# Future roll scenario : Maturity Approach - C#

### Overview
Future contracts are liquid and often used to gain exposure to a variety of asset classes. However they must be rolled on a periodic basis to avoid expiry or triggering delivery.

The period surrounding contract rollover can be challenging time. Future roll strategies help to better prepare moves in the market and optimize trading positions.

In order to identify future roll optimal dates, the following methods can be used:
1. Maturity date
2. Trading volume
3. Market activity

In this example, the strategy chosen to determine the future roll date  is based on the **maturity date**.

This sample demonstrates how to request and plot from a dedicated data service **on-demand** bars for a <i>`given future, all maturities`</i>.

### Inputs/outputs
Future roll sample requires future contact's identifier and date time intervals as per inputs. It returns future's continuous price over the look back period.<br>
The results is sampled using a one day delay until front's maturity to roll from front to back. 

This sample shows also how to plot the returned bars using an open source library.
### Services used
This sample uses *gRPC request*s in order to retrieve information from hosted services. The queried endpoint in this script are:
* *StaticDataService*: to get the intrument identifier from the input
* *TopologiesService*: to directly retrieve ticks objects from the server.
* *TickBarsService*: to get tick bars data for each instrument

### Packages required
1. Systemathics packages:
    * *Systemathics.Apis.Type.Shared.V1*
    * *Systemathics.Apis.Services.StaticData.V1*
    * *Systemathics.Apis.Services.TickAnalytics.V1*
2. Open source packages
    * *Google.Protobuf.WellKnownTypes*
    * *Google.Type*
    * *Grpc.Net.Client*
    * *Grpc.Core*
    * *XPlot.Plotly* as per display package

***

# Run future roll sample - maturity approach

### Step 1: Install packages

In [ ]:
#i "nuget: file:///home/jovyan/.nuget/packages/"
#i "nuget: https://api.nuget.org/v3/index.json"
#r "nuget: Systemathics.Apis"
#r "nuget: XPlot.Plotly.Interactive"

In [ ]:
using Systemathics.Apis.Helpers;
using Systemathics.Apis.Type.Shared.V1;
using Systemathics.Apis.Services.TickAnalytics.V1;
using Systemathics.Apis.Services.StaticData.V1;
using Google.Protobuf.WellKnownTypes;
using Google.Protobuf;
using Google.Type;
using Grpc.Net.Client;
using Grpc.Core;
using System.Collections.Generic;
using XPlot.Plotly;
using XPlot.Plotly.Interactive;

### Step 2: Prepare API requests
The following code snippets retrieve authentication token and prepare the API request by: opening the *channel* and adding the *token* to the request header:

In [ ]:
// Get token as metadata
var headers = TokenHelpers.GetTokenAsMetaData();

// Create communication channel
var channel = ChannelHelpers.GetChannel();

### Step 3: Select futures scope

#### 3.1 Define inputs
In a first time the user has to decide the lookup time span and the selected future contract.

In [ ]:
// Define start, end and future contract
var start = new DateTime(2020,02, 01);
var end = new DateTime(2020, 07, 31);
var contract = "WBS"; // WTI Crude future

#### 3.2 Select future contracts

In [ ]:
// instantiate the static data service
var service = new StaticDataService.StaticDataServiceClient(channel);

In [ ]:
// Get futures from static data request
// We chose to retrieve the selected instrument with its contract code.
var request = new StaticDataRequest
{
    AssetType = AssetType.Future,
    FutureContract = contract,
    Count = 1000
};

// store futures and display results
var reply = service.StaticData(request, headers);
var futures = reply.Futures.OrderBy(future => future.Maturity.Year).ThenBy(future => future.Maturity.Month); 

In [ ]:
// Display results
//display(futures)

#### 3.3 Filter futures matching input period

In [ ]:
// Filter futures : select only futures with a maturity matching the selected period
var tmpList = new List<FutureEntry>();
var lastFuture = true; // we need this boolean to select one more future
foreach (var future in futures) {
    var maturity = new DateTime(future.Maturity.Year,future.Maturity.Month, future.Maturity.Day);
    if (maturity > start) 
    {
        if (maturity < end) {
            tmpList.Add(future);
        } 
        else 
        {
            if (lastFuture == true) 
            {
                tmpList.Add(future); // add the last contract
                lastFuture = false;
            } 
            else {
                break;
            }
        }
    }
}
var count = tmpList.Count;
var selectedFutures = tmpList.ToArray();

In [ ]:
// Display previously selected futures
// display(selectedFutures)

#### 3.4 Prepare bars request parameters
Select start date, end date and instrument identifier for the previsouly selected futures.

In [ ]:
// Create a list of tuples containing the instrument identifier and date intervals for bars request
var barsRequestData = new List<Tuple<Identifier, DateTime,DateTime>>();
var beginDate = start;
var endDate = start;

var daysDelta = 1; // The days delta chosen here defines the delay until expiration before rolling

for (int i = 0; i < count; i++)
{
    var currentFuture = selectedFutures[i];
    var maturity = new DateTime(currentFuture.Maturity.Year,currentFuture.Maturity.Month, currentFuture.Maturity.Day);
    
    // We have to deal with the first (resp. last) futures differently since the start (resp. end) dates will be determined by the given time period
    if (i == 0) {
        // handle first future: specific beginDate
        beginDate = start;
        endDate = maturity.AddDays(-daysDelta);
        var result = new Tuple<Identifier, DateTime,DateTime>(currentFuture.Identifier, beginDate, endDate);
        barsRequestData.Add(result);
        
    } 
    else if (i == count-1) 
    {
        // handle last future: specific endDate
        beginDate = endDate.AddDays(1); // set the new beginDate for that future to the previous endDate + one day
        endDate = end;
        var result = new Tuple<Identifier, DateTime,DateTime>(currentFuture.Identifier, beginDate, endDate);
        barsRequestData.Add(result);
    } 
    else 
    {
        // default case
        beginDate = endDate.AddDays(1); // set the new beginDate for that future to the previous endDate + one day
        endDate = maturity.AddDays(-daysDelta);
        var result = new Tuple<Identifier, DateTime,DateTime>(currentFuture.Identifier, beginDate, endDate);
        barsRequestData.Add(result);
    }
    
}

In [ ]:
// Display bars request parameters
//display(barsRequestData)

### Step 4: Get bars to get a continuous price
The previous bars request parameters ensure to get continious prices over the chosen look back period.

#### 4.1 Define bar request creation method
The following code snippt defines a method that creates and handles *tick bars* requests: 

In [ ]:
public static TickBarsRequest GetBarsRequest(Identifier identifier, DateTime start, DateTime end)
{
    // Set the bar duration (1 hour bars in seconds)
    var sampling = 60 * 60;

    // Set the bar calculation field (Trade price)
    var field = BarPrice.Trade; 

    // Begin
    var beginTime = new TimeOfDay { Hours = 00, Minutes = 00, Seconds = 00 };

    // Create bars request data intervals
    var dateIntervals = new DateInterval()
      {
        StartDate = new Date { Year = start.Year, Month = start.Month, Day = start.Day },
        EndDate = new Date { Year = end.Year, Month = end.Month, Day = end.Day }

      };
    
    // Generate constraints based on the previous date selection
    var constraints = new Constraints(); 
    constraints.DateIntervals.Add(dateIntervals);
    
    // Generate tick bars request
    var request = new TickBarsRequest 
    {
        Identifier = identifier, 
        Constraints = constraints,  
        Sampling = new Duration {Seconds = sampling},
        Field = field
    };
    return request;
}

#### 4.2 Process bars request for each future
We have to use the previously defined  `GetBarsRequest` method to generate the request and process it. Results will be stored in a dictionary before being displayed.

In [ ]:
// Intiate dictionary to handle bars data for each future contract
var barsResponses = new Dictionary<string, List<TickBarsResponse>>();

// Instantiate the tick bars service
var barsService = new TickBarsService.TickBarsServiceClient(channel);

// Process the tick bars reply: store tick bars for each future contract
foreach (var elt in barsRequestData)
{
    // Generate tick bars request
    var barsRequest = GetBarsRequest(elt.Item1, elt.Item2, elt.Item3);
    
    var key = elt.Item1.Ticker;
    barsResponses[key] = new List<TickBarsResponse>();
    
    var call = barsService.TickBars(barsRequest, headers);
    await foreach (var bar in call.ResponseStream.ReadAllAsync())
    {
        barsResponses[key].Add(bar);
    }
}

### Step 5: Visualize data

The following code snippets retrieve for each maturity, the *Close prices* over the look back period from the request result and plot them with the package of your choice.<br><i>`XPlot.Plotly`</i> is used as per open source display package.

In [ ]:
// Define a method transforming a dictionary key/value pair to a graph content
public static Scattergl ToGraph(KeyValuePair<string, List<TickBarsResponse>> kvp)
{
    return new Scattergl
    {
        name = kvp.Key,
        x = kvp.Value.Select(b => b.TimeStamp.ToDateTime()).ToArray(),
        y = kvp.Value.Select(b => b.Close).ToArray(),
    };
}

In [ ]:
var chartContent = barsResponses.Select(ToGraph).ToArray();
var chart = Chart.Plot(chartContent);
var layout = new Layout.Layout(){barmode = "group" };
chart.WithLayout(layout);
chart.WithTitle($"{contract} roll using maturity dates | 1-hour bars [{start:yyyy-MM-dd} - {end:yyyy-MM-dd}] ");
chart.WithXTitle("Dates");
chart.WithYTitle("Close price");
chart.WithLegend(true);
chart.Width = 1500;
chart.Height = 800;
display(chart);